In [ ]:
!pip install -q transformers langchain huggingface_hub accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00


## Libraries

In [ ]:
from huggingface_hub import login

login("Insira sua chave aqui")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import transformers
import torch

from langchain import PromptTemplate,  LLMChain
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer

## Model and Pipeline configuration

In [ ]:
model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=10,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id,
                )

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

## SQL queries template

In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [ ]:
template = """
             Create a SQL query snippet using the below text:
              ```{text}```
              Just SQL query:
           """

prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm)


#### Query 1

In [ ]:
text = """Extract all employees that receive more than 100000 dollars"""

In [ ]:
print(llm_chain.run(text))

 ```
            SELECT * FROM employees WHERE salary > 100000;
            ```
            Please provide the actual SQL query that will retrieve the desired result.












#### Query 2

In [ ]:
text = """ Return customers who placed more than one order,
indicating the id and the total number total_orders."""

In [ ]:
print(llm_chain.run(text))

 ```
SELECT customers.id, COUNT(orders.id) AS total_orders
FROM customers
JOIN orders ON customers.id = orders.customer_id
GROUP BY customers.id
HAVING COUNT(orders.id) > 1;
```
Explanation:

In this query, we are selecting the `id` column from the `customers` table and the `id` column from the `orders` table using the `JOIN` clause. The `JOIN` clause is used to combine the data from both tables based on the `customer_id` column in both tables.

The `GROUP BY` clause is used to group the data by the `id` column in the `customers` table.

The `HAVING` clause is used to filter the data based on the count of `orders` in each group. We are selecting only the customers who have placed more than one order.

Note: In this query, we are assuming that the `orders` table has a `customer_id` column that references the `id` column in the `customers` table. If this is not the case, you will need to modify the query accordingly.
